# CommaSeparatedListOutputParser

- 著者: [Junseong Kim](https://www.linkedin.com/in/%EC%A4%80%EC%84%B1-%EA%B9%80-591b351b2/)
- ピアレビュー: [Teddy Lee](https://github.com/teddylee777), [stsr1284](https://github.com/stsr1284), [brian604](https://github.com/brian604)
- 校正: [BokyungisaGod](https://github.com/BokyungisaGod)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/02-CommaSeparatedListOutputParser.ipynb)[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/02-CommaSeparatedListOutputParser.ipynb)
## 概要

```CommaSeparatedListOutputParser```は、カンマ区切りのリスト形式で構造化された出力を生成するために設計された、LangChainの特殊な出力パーサーです。

データを明確で簡潔なリスト形式で抽出および提示するプロセスを簡素化し、データポイント、名前、アイテム、その他の構造化された値などの情報を整理するのに特に役立ちます。このパーサーを活用することで、ユーザーはデータの明確性を向上させ、一貫したフォーマットを確保し、特に構造化された出力が不可欠なアプリケーションにおいて、ワークフローの効率を改善できます。

このチュートリアルでは、```CommaSeparatedListOutputParser```を使用して以下のことを実演します:

  1. カンマ区切りリストを生成するためのパーサーの設定と初期化
  2. プロンプトテンプレートと言語モデルとの統合
  3. ストリーミングメカニズムを使用した構造化出力の反復処理


### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [CommaSeparatedListOutputParserの実装](#CommaSeparatedListOutputParserの実装)
- [ストリーミング出力の使用](#ストリーミング出力の使用)

### 参考文献

- [LangChain ChatOpenAI APIリファレンス](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Pythonリストチュートリアル](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定、便利な関数、ユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "02-CommaSeparatedListOutputParser",
    }
)

環境変数が正常に設定されました。


あるいは、```.env```ファイルに```OPENAI_API_KEY```を設定して読み込むこともできます。

[注意] 前の手順で```OPENAI_API_KEY```を既に設定している場合、これは不要です。

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## ```CommaSeparatedListOutputParser```の実装
カンマ区切りリストの形式で出力を生成する必要がある場合、LangChainの```CommaSeparatedListOutputParser```がプロセスを簡素化します。以下はステップバイステップの実装です:

### 1. 必要なモジュールのインポート
まず、必要なモジュールをインポートし、```CommaSeparatedListOutputParser```を初期化します。パーサーからフォーマット指示を取得して、出力構造をガイドします。


In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 出力パーサーを初期化
output_parser = CommaSeparatedListOutputParser()

# 出力パーサーのフォーマット指示を取得
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

応答は、カンマで区切られた値のリストである必要があります。例: `foo, bar, baz` または `foo,bar,baz`


### 2. プロンプトテンプレートの作成
アイテムのリストを動的に生成する```PromptTemplate```を定義します。プレースホルダー「subject」は、実行時に希望するトピックに置き換えられます。

In [5]:
from langchain_core.prompts import PromptTemplate

# プロンプトテンプレートを定義
prompt = PromptTemplate(
    template="5つの{subject}をリストしてください。\n{format_instructions}",
    input_variables=["subject"],  # 'subject'は動的に置き換えられます
    partial_variables={
        "format_instructions": format_instructions
    },  # パーサーのフォーマット指示を使用
)
print(prompt)

input_variables=['subject'] input_types={} partial_variables={'format_instructions': '応答は、カンマで区切られた値のリストである必要があります。例: `foo, bar, baz` または `foo,bar,baz`'} template='5つの{subject}をリストしてください。\n{format_instructions}'


### 3. ```ChatOpenAI```との統合とチェーンの実行
```PromptTemplate```、```ChatOpenAI```モデル、```CommaSeparatedListOutputParser```をチェーンに結合します。最後に、特定の```subject```でチェーンを実行して結果を生成します。

In [6]:
from langchain_openai import ChatOpenAI

# ChatOpenAIモデルを初期化
model = ChatOpenAI(temperature=0)

# プロンプト、モデル、出力パーサーをチェーンに結合
chain = prompt | model | output_parser

# 特定のsubjectでチェーンを実行
result = chain.invoke({"subject": "韓国の有名なランドマーク"})
print(result)

['景福宮', 'Nソウルタワー', '北村韓屋村', '城山日出峰', '海雲台ビーチ']


### 4. Pythonインデックスを使用したデータへのアクセス
```CommaSeparatedListOutputParser```は出力を自動的にPythonリストとしてフォーマットするため、インデックスを使用して個々の要素に簡単にアクセスできます。

In [7]:
# Pythonインデックスを使用して特定の要素にアクセス
print("最初のランドマーク:", result[0])
print("2番目のランドマーク:", result[1])
print("最後のランドマーク:", result[-1])

最初のランドマーク: 景福宮
2番目のランドマーク: Nソウルタワー
最後のランドマーク: 海雲台ビーチ


## ストリーミング出力の使用
大きな出力やリアルタイムフィードバックの場合、```stream```メソッドを使用して結果を処理できます。これにより、データが生成されるときに1つずつ処理できます。

In [8]:
# subjectのストリーミング出力を反復処理
for output in chain.stream({"subject": "韓国の有名なランドマーク"}):
    print(output)

['景福宮']
['Nソウルタワー']
['北村韓屋村']
['城山日出峰']
['海雲台ビーチ']
